<a href="https://colab.research.google.com/github/A1exanderBates/GoogleBooksAPI-to-BigQuery/blob/main/Google_Books_to_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries
import requests
import json
import csv

# creating a list of genres for the dynamic search parameter
genre = ["Art",
         "Biography",
         "Business",
         "Children's",
         "Christian",
         "Classics",
         "Comics",
         "Cookbooks",
         "Fantasy",
         "Fiction",
         "Graphic Novels",
         "Historical Fiction",
         "History",
         "Horror",
         "Memoir",
         "Music",
         "Mystery",
         "Nonfiction",
         "Poetry",
         "Psychology",
         "Romance",
         "Science",
         "Science Fiction",
         "Self Help",
         "Sports",
         "Thriller",
         "Travel",
         "Young Adult"]


raw_results = []

# retrieving Google Books volume data with dynamic search parameter
for i in genre:
    response_data = requests.get('https://www.googleapis.com/books/v1/volumes?q=' + i + '&startIndex=0&maxResults=40').json()
    raw_results.append(response_data)

# parsing through the retrieved dictionary data 
retrieved_items_data = []
for i in range(0, len(raw_results)):
    retrieved_items_data.append(raw_results[i]['items'])

# further parsing to retrieve volumeInfo
volumeInfo = []
for k in range(0, len(retrieved_items_data)):
    by_genre_record = retrieved_items_data[k]
    for i in range(0,len(by_genre_record)):
        temp_dict1 = by_genre_record[i]['volumeInfo']
        volumeInfo.append(temp_dict1)

# writing the extracted data to a csv file
volume_col_headers = ['allowAnonLogging', 
                        'authors',
                        'averageRating', 
                        'canonicalVolumeLink',
                        'categories',
                        'contentVersion',
                        'description',
                        'imageLinks',
                        'thumbnail',
                        'industryIdentifiers',
                        'infoLink',
                        'language',
                        'maturityRating',
                        'pageCount',
                        'panelizationSummary',
                        'containsImageBubbles',
                        'previewLink',
                        'printType',
                        'publishedDate',
                        'publisher',
                        'ratingsCount',
                        'readingModes',
                        'subtitle',
                        'title',
                        'comicsContent',
                        'seriesInfo']
with open('/tmp/books_API_volume_data.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=volume_col_headers)
        writer.writeheader()
        writer.writerows(volumeInfo)

# Creating a function to push CSV file to Cloud Storage Bucket
def push_to_gcs(file, bucket):
    file_name = file.split('/')[-1]
    print(f"Pushing {file_name} to GCS...")
    blob = bucket.blob(file_name)
    blob.upload_from_filename(file)
    print(f"File pushed to {blob.id} succesfully.")

file_name = 'books_API_volume_data.csv' # This is your csv file created in step 5
file_path = '/tmp/' + file_name

# Move csv file to Cloud Storage
storage_client = storage.Client()
bucket_name = 'gbooks-portfolio-bucket' # This is your cloud storage bucket
bucket = storage_client.get_bucket(bucket_name)
push_to_gcs(file_path, bucket)
    